# Maximum Entropy Matching Distribution Function

This notebook can construct the distribution function which maximizes the kinetic entropy subject to the constraints of matching the energy density $\epsilon$ and the shear stress $\pi^{\mu\nu}$.

The entropy density current is expressed

$$s^{\mu}(x) = - \sum_h \frac{g_h}{(2\pi)^3}\int \frac{d^3p}{p_0} p^{\mu} \phi[f_h],$$

where $h$ runs over all hadronic species, $g_h$ is the degeneracy of species $h$, and $f_h(x;p)$ is the distribution function of particles of species $h$.

The function $\phi[f_h]$ defined by

$$ \phi[f] \equiv f \ln (f) - \frac{1 + \theta f}{\theta} \ln(1 + \theta f)$$

defines the quantum statistical nature of the particles, with $\theta = 1$, $0$ or $-1$ for bosons, Maxwell-Boltzmann particles, and fermions respectively.

In [24]:
%load_ext Cython
from scipy import integrate
from scipy import interpolate
from scipy.interpolate import RectBivariateSpline
from scipy import optimize
from numpy import linalg as LA
from mpmath import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dill
import pandas as pd
import time
sns.set()
sns.set_style('whitegrid')
sns.set_context('talk')

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [25]:
#These triple series expressions for epsilon, pi_1 and pi_2 
#are still slow for phenomenology
#alternatively, the integral expression for the generating 
#function could be used

hbar = 0.197
#hbar = 1. #temporary

sum_max = 10

method_sum='alternating'
#method_sum='direct'

gamma = memoize(gamma) #speed up mpmath

#the angular function
def angular(nx, ny, nz):
    ns = nx+ny+nz
    a = (1+(-1)**(2*nx))*(1+(-1)**(2*ny))*(1+(-1)**(2*nz))/4.
    b = gamma((1.+ (2.*nx))/2.) * gamma((1.+ (2.*ny))/2.) * gamma((1.+ (2.*nz))/2.) / gamma((3.+(2.*ns))/2.)
    return mpmathify(a*b)

angular = memoize(angular) #speed up mpmath

def sum_e_fct(beta, g1, g2):
    beta = mpmathify(beta)
    g1 = mpmathify(g1)
    g2 = mpmathify(g2)
    g3 = -(g1+g2)
    def e_fct(nx, ny, nz):
        nx = mpmathify(nx)
        ny = mpmathify(ny)
        nz = mpmathify(nz)
            
        ns = nx + ny + nz
        a = angular(nx, ny, nz) * gamma(3+ns) / (fac(nx)*fac(ny)*fac(nz))
        b = ((-g1/beta)**nx) * ((-g2/beta)**ny) * ((-g3/beta)**nz)
        c = (3+ns)
        pre_fac = 1./ ((2.*np.pi*hbar)**3.) / (beta ** 4.)
        pre_fac = mpmathify(pre_fac)
        return pre_fac*a*b*c
    return float(nsum( e_fct, [0, sum_max], [0, sum_max], [0, sum_max], method=method_sum ))
        
def sum_pi_1_fct(beta, g1, g2):
    beta = mpmathify(beta)
    g1 = mpmathify(g1)
    g2 = mpmathify(g2)
    g3 = -(g1+g2)
    def pi_1_fct(nx, ny, nz):
        nx = mpmathify(nx)
        ny = mpmathify(ny)
        nz = mpmathify(nz)
            
        ns = nx + ny + nz
        a = (-1)**(ns) * angular(nx, ny, nz) * gamma(3+ns) / (fac(nx) * fac(ny) * fac(nz))
        b = ((g1/beta)**nx) * ((g2/beta)**ny) * ((g3/beta)**nz)
        c = ( (3.+ns)/3. + (nx*beta/g1) )
        pre_fac = -1./ ((2.*np.pi*hbar)**3.) / (beta ** 4.)
        pre_fac = mpmathify(pre_fac)
        return pre_fac*a*b*c 
    return float(nsum( pi_1_fct, [0, sum_max], [0, sum_max], [0, sum_max], method=method_sum ))

def sum_pi_2_fct(beta, g1, g2):
    beta = mpmathify(beta)
    g1 = mpmathify(g1)
    g2 = mpmathify(g2)
    g3 = -(g1+g2)
    def pi_2_fct(nx, ny, nz):
        nx = mpmathify(nx)
        ny = mpmathify(ny)
        nz = mpmathify(nz)
            
        ns = nx + ny + nz
        a = (-1)**(ns) * angular(nx, ny, nz) * gamma(3+ns) / (fac(nx) * fac(ny) * fac(nz))
        b = ((g1/beta)**nx) * ((g2/beta)**ny) * ((g3/beta)**nz)
        c = ( (3.+ns)/3. + (ny*beta/g2) )
        pre_fac = -1./ ((2.*np.pi*hbar)**3.) / (beta ** 4.)
        pre_fac = mpmathify(pre_fac)
        return pre_fac*a*b*c 
    return float(nsum( pi_2_fct, [0, sum_max], [0, sum_max], [0, sum_max], method=method_sum ))

In [31]:
#the equilibrium temperature (EoS)
def equil_beta(eps):
    a = 3. / (eps * np.pi**2. * hbar**3.)
    return a**(0.25)

def beta_pi(eps):
    beta = equil_beta(eps)
    I_1_42 = 4. / (5. * np.pi**2. * hbar**3. * beta**5.)
    return beta * I_1_42

def f_CE(p, e, pi_ij):
    beta = equil_beta(e)
    beta_pi = beta_pi(e)
    p_0 = np.sqrt(p@p) #energy for massless case
    f_eq = np.exp(-beta*p_0)
    ang_fac = p @ pi_ij @ p
    df = beta * f_eq * ang_fac / (2. * p_0 * beta_pi)

In [32]:
#this root solving routine is still slow,
#but can probably be sped up with a c backend 
#and some algebraic re-arranging of the various terms
#Sage has a cython mpmath backend (https://www.sagemath.org)

def root_solve_beta_gamma(e0, pi_1, pi_2, tol=1e-4, maxiter = int(1e5)):    
    
    def my_fun(x):
        beta = x[0]
        g1 = x[1]
        g2 = x[2]
        e_val = sum_e_fct(beta, g1, g2)
        pi_1_val = sum_pi_1_fct(beta, g1, g2)
        pi_2_val = sum_pi_2_fct(beta, g1, g2)
        return [e_val - e0, pi_1_val - pi_1, pi_2_val - pi_2]
    
    def print_warning(e0, pi_1, pi_2):
        print("WARNING - NO ROOT FOUND: e0 = " + str(e0) + " ; pi_1 = " + str(pi_1))
        
    #USE Linear CE RTA to initialize the rootsolver
    beta_guess = equil_beta(e0)
    beta_pi_guess = beta_pi(e0)
    guess_fac = -beta_guess / (2. * beta_pi_guess)
    g1_guess = guess_fac * pi_1 
    g2_guess = guess_fac * pi_2 
    guess = [beta_guess, g1_guess, g2_guess]
    print("guess = " + str(guess))
    
    solution = optimize.root(my_fun, guess, tol=tol)
    #solution = optimize.root(my_fun, guess, method=method_root, tol=tol, options={'maxiter':maxiter})
    
    if (solution.success == False):
        print_warning(e0, pi_1, pi_2)
        
    beta_soln = solution.x[0]
    g1_soln = solution.x[1]
    g2_soln = solution.x[2]
    
    return beta_soln, g1_soln, g2_soln, solution

In [33]:
#Checks that the functions are working correctly
t1 = time.time()
e = sum_e_fct(beta=1., g1=0.1, g2=0.2)
t2 = time.time()
pi_1 = sum_pi_1_fct(beta=1., g1=0.1, g2=0.2)
t3 = time.time()
pi_2 = sum_pi_2_fct(beta=1., g1=0.1, g2=0.2)
print("e = " + str(e) + " ?=? 49.772")
print("pi_1 = " + str(pi_1) + " ?=? -4.035")
print("pi_2 = " + str(pi_2) + " ?=? -5.5907")
print("e sum eval'd in " + str(t2-t1) + " sec")
print("pi_1 sum eval'd in " + str(t3-t2) + " sec")

e = 49.77232274243643 ?=? 49.772
pi_1 = -4.035753999037346 ?=? -4.035
pi_2 = -5.5907058966960825 ?=? -5.5907
e sum eval'd in 0.1926417350769043 sec
pi_1 sum eval'd in 0.18334007263183594 sec


In [34]:
#Check the root solving method
root_solve_beta_gamma(e, pi_1, pi_2, tol=1e-7, maxiter = int(1e5))

guess = [0.9453850507168635, 0.1437297862862133, 0.19910801399512276]


(1.0000000000095055,
 0.10000000004144453,
 0.1999999999266035,
     fjac: array([[-0.99166623, -0.1145728 , -0.05891655],
        [ 0.12837433, -0.91733174, -0.37685874],
        [ 0.01086826,  0.38128146, -0.92439512]])
      fun: array([-5.17123766e-09,  1.72271974e-10,  2.01718109e-09])
  message: 'The solution converged.'
     nfev: 13
      qtf: array([-2.45084010e-07,  9.76321971e-08,  7.61172764e-08])
        r: array([104.88686408, -57.89525924, -87.95218487,  43.7755015 ,
         46.55294486,  25.59264355])
   status: 1
  success: True
        x: array([1. , 0.1, 0.2]))

In [35]:
def find_shear_eig(pi_ij):
    """pi_ij should be the shear stress tensor in the LRF"""
    w, v = LA.eig(pi_ij)
    return w, v

In [36]:
#this function solves for all the lagrange multipliers
#and returns them
def find_lagrange_multipliers(e, pi_xx, pi_xy, pi_xz, pi_yy, pi_yz):

    pi_zz = -(pi_xx + pi_yy) #traceless

    #symmetric matrix
    pi_ij = np.array( [[pi_xx, pi_xy, pi_xz],[pi_xy, pi_yy, pi_yz],[pi_xz, pi_yz, pi_zz]] )

    w, v = find_shear_eig(pi_ij) #w are eig vals, v is matrix of eig vectors

    pi_1, pi_2 = w[0], w[1] #first two eig vals

    #now root solve to find lagrange multipliers
    beta_soln, g1_soln, g2_soln, info = root_solve_beta_gamma(e, pi_1, pi_2, tol=1e-7, maxiter = int(1e5))

    g3_soln = -(g1_soln + g2_soln)
    #construct the diagonal gamma matrix
    gamma_ij_D = np.array( [[g1_soln, 0., 0.],[0., g2_soln, 0.],[0., 0., g3_soln]] )
    #now rotate gamma out of the eigenvector basis
    gamma_ij = (v.T)@(gamma_ij_D)@(v)
    
    return beta_soln, gamma_ij, info

In [39]:
#the ME distribution
def f_dist(p, e, pi_xx, pi_xy, pi_xz, pi_yy, pi_yz):
    """Input assumes p is a three-vector momentum."""
    beta, gamma, info = find_lagrange_multipliers(e, pi_xx, pi_xy, pi_xz, pi_yy, pi_yz)
    p_0 = np.sqrt(p@p) #energy for massless case
    a = np.exp(-beta*p_0)
    b = np.exp(-p@gamma@p / p_0)
    return a*b, info

In [41]:
e = 50.
pi_xx = -e/10.
pi_yy = -e/10.
pi_xy = 0.
pi_xz = 0.
pi_yz = 0.
pi_zz = -(pi_xx + pi_yy)

pi_ij = np.array( [[pi_xx, pi_xy, pi_xz],[pi_xy, pi_yy, pi_yz],[pi_xz, pi_yz, pi_zz]] )

p = np.array([1., 1., 1.])

f_dist(p, e, pi_xx, pi_xy, pi_xz, pi_yy, pi_yz)

guess = [0.9443069947218412, 0.1770575615103452, 0.1770575615103452]


(0.17661816756910934,     fjac: array([[-0.98956165,  0.10190098,  0.10190155],
        [-0.13210209, -0.9240042 , -0.35883878],
        [-0.05759143,  0.36855451, -0.92782046]])
      fun: array([-7.81703662e-08,  2.39670479e-08,  2.39672273e-08])
  message: 'The solution converged.'
     nfev: 12
      qtf: array([ 4.54191596e-07, -1.12782495e-07, -4.91686629e-08])
        r: array([ 202.80039522, -112.39050903, -112.38968675,   35.19113573,
          23.95049053,   25.78321121])
   status: 1
  success: True
        x: array([1.00098976, 0.15381181, 0.15381181]))